In [14]:
# Data visualization
import numpy as np
import pandas as pd 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt  # This import is fine



# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Important Landmarks and Important functions

In [15]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [4]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

## 2. Describe Dataset & Split Data

In [5]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 15372 
Number of columns: 37

Labels: 
label
C    8238
L    7134
Name: count, dtype: int64

Missing values: False

Duplicate Rows : 0


In [16]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [17]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

## 3. Build Model

### 3.1. Set up

In [19]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [20]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

### 3.2. Model with 3 layers

In [21]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [22]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 06s]
val_accuracy: 0.9954471588134766

Best val_accuracy So Far: 0.9980487823486328
Total elapsed time: 00h 01m 39s


In [32]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000001F7DFC09800>
Layer-2: 32 units, func:  <function relu at 0x000001F7DFC09800>
Layer-3: 2 units, func:  <function softmax at 0x000001F782990C20>

Other params:
learning_rate: 0.001
Epoch 1/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 960us/step - accuracy: 0.9408 - loss: 0.1597 - val_accuracy: 0.9932 - val_loss: 0.0215
Epoch 2/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9923 - loss: 0.0273 - val_accuracy: 0.9967 - val_loss: 0.0136
Epoch 3/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9928 - loss: 0.0223 - val_accuracy: 0.9948 - val_loss: 0.0168


In [33]:
final_models["3_layers"] = model_3l

### 3.3. Model with 5 layers

In [25]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [26]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
tuner_5l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 12s]
val_accuracy: 0.9970731735229492

Best val_accuracy So Far: 0.9983739852905273
Total elapsed time: 00h 02m 54s


In [27]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000001F7DFC09800>
Layer-2: 256 units, func:  <function relu at 0x000001F7DFC09800>
Layer-3: 352 units, func:  <function relu at 0x000001F7DFC09800>
Layer-4: 32 units, func:  <function relu at 0x000001F7DFC09800>
Layer-5: 2 units, func:  <function softmax at 0x000001F782990C20>

Other params:
learning_rate: 0.001
Epoch 1/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9685 - loss: 0.0949 - val_accuracy: 0.9951 - val_loss: 0.0171
Epoch 2/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9939 - loss: 0.0236 - val_accuracy: 0.9954 - val_loss: 0.0127
Epoch 3/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9940 - loss: 0.0190 - val_accuracy: 0.9909 - val_loss: 0.0255
Epoch 4/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9951 - loss: 0.0166 - val_accuracy: 0.9958 - val_loss: 0.0128
Epoch 5/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9958 - loss: 0.014

In [28]:
final_models["5_layers"] = model_5l

### 3.4. Model with 7 layers include Dropout

In [29]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [30]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 08s]
accuracy: 0.9822720885276794

Best accuracy So Far: 0.9974790811538696
Total elapsed time: 00h 03m 31s


In [31]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000001F7DFC09800>
Layer-2: 192 units, func:  <function relu at 0x000001F7DFC09800>
Layer-3: 0 units, func: None
Layer-4: 64 units, func:  <function relu at 0x000001F7DFC09800>
Layer-5: 0 units, func: None
Layer-6: 64 units, func:  <function relu at 0x000001F7DFC09800>
Layer-7: 2 units, func:  <function softmax at 0x000001F782990C20>

Other params:
dropout_1: 0.2
dropout_2: 0.2
learning_rate: 0.001
Epoch 1/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9409 - loss: 0.1506 - val_accuracy: 0.9935 - val_loss: 0.0239
Epoch 2/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9929 - loss: 0.0285 - val_accuracy: 0.9961 - val_loss: 0.0115
Epoch 3/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9937 - loss: 0.0250 - val_accuracy: 0.9967 - val_loss: 0.0151
Epoch 4/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9930 - loss: 0.0196 - val_accuracy: 0.9974 - val_loss: 0.0088


In [34]:
final_models["7_layers_with_dropout"] = model_7lD

### 3.5. Model with 7 layers

In [35]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [36]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 08s]
val_accuracy: 0.9915447235107422

Best val_accuracy So Far: 0.9980487823486328
Total elapsed time: 00h 04m 00s


In [38]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000001F7DFC09800>
Layer-2: 416 units, func:  <function relu at 0x000001F7DFC09800>
Layer-3: 288 units, func:  <function relu at 0x000001F7DFC09800>
Layer-4: 320 units, func:  <function relu at 0x000001F7DFC09800>
Layer-5: 480 units, func:  <function relu at 0x000001F7DFC09800>
Layer-6: 256 units, func:  <function relu at 0x000001F7DFC09800>
Layer-7: 2 units, func:  <function softmax at 0x000001F782990C20>

Other params:
learning_rate: 0.0001
Epoch 1/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9476 - loss: 0.1612 - val_accuracy: 0.9928 - val_loss: 0.0266
Epoch 2/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9927 - loss: 0.0253 - val_accuracy: 0.9961 - val_loss: 0.0151
Epoch 3/100
1230/1230 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9966 - loss: 0.0133 - val_accuracy: 0.9922 - val_loss: 0.0223


In [39]:
final_models["7_layers"] = model_7l

### 3.6. Describe final models

In [40]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

5_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000001F7DFC09800>
Layer-2: 256 units, func:  <function relu at 0x000001F7DFC09800>
Layer-3: 352 units, func:  <function relu at 0x000001F7DFC09800>
Layer-4: 32 units, func:  <function relu at 0x000001F7DFC09800>
Layer-5: 2 units, func:  <function softmax at 0x000001F782990C20>

3_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000001F7DFC09800>
Layer-2: 32 units, func:  <function relu at 0x000001F7DFC09800>
Layer-3: 2 units, func:  <function softmax at 0x000001F782990C20>

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x000001F7DFC09800>
Layer-2: 192 units, func:  <function relu at 0x000001F7DFC09800>
Layer-3: 0 units, func: None
Layer-4: 64 units, func:  <function relu at 0x000001F7DFC09800>
Layer-5: 0 units, func: None
Layer-6: 64 units, func:  <function relu at 0x000001F7DFC09800>
Layer-7: 2 units, func:  <function 

## 4. Model Evaluation

### 4.1. Train set evaluation

In [41]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,5_layers,"[0.997, 0.999]","[0.999, 0.996]","[0.998, 0.997]","[[1677, 2], [5, 1391]]"
1,7_layers_with_dropout,"[0.997, 0.997]","[0.998, 0.996]","[0.997, 0.997]","[[1675, 4], [5, 1391]]"
2,3_layers,"[0.998, 0.991]","[0.992, 0.998]","[0.995, 0.994]","[[1666, 13], [3, 1393]]"
3,7_layers,"[0.999, 0.984]","[0.986, 0.999]","[0.993, 0.991]","[[1656, 23], [1, 1395]]"


### 4.2. Test set evaluation

In [42]:
# load dataset
test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 604 
Number of columns: 37

Labels: 
label
C    339
L    265
Name: count, dtype: int64

Missing values: False

Duplicate Rows : 0


In [43]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [44]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers,"[0.996, 0.786]","[0.788, 0.996]","[0.88, 0.879]","[[267, 72], [1, 264]]"
1,7_layers,"[0.944, 0.928]","[0.944, 0.928]","[0.944, 0.928]","[[320, 19], [19, 246]]"
2,7_layers_with_dropout,"[0.844, 0.985]","[0.991, 0.766]","[0.912, 0.862]","[[336, 3], [62, 203]]"
3,5_layers,"[0.762, 1.0]","[1.0, 0.6]","[0.865, 0.75]","[[339, 0], [106, 159]]"


## 5. Dumped Model

In [45]:
# Dump the best model to a pickle file
with open("./model/bicep_dp.pkl", "wb") as f:
    pickle.dump(final_models["7_layers"], f)

In [46]:
# Dump final results
with open("./model/all_models.pkl", "wb") as f:
    pickle.dump(final_models, f)